In [2]:
import pandas as pd
# import numpy as np


df = pd.read_csv('rating.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [3]:
# df.info(memory_usage="deep")
# df.memory_usage()
# df.describe()

In [4]:
# parse the timestamp because it's currently an object
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Double check it's been parse correctly
df.timestamp.dtype

dtype('<M8[ns]')

In [5]:
# Calculate load per year and reset index. "A" is annual
load_per_year = df.groupby(pd.Grouper(key="timestamp", freq="A")).count()
load_per_year.reset_index(inplace=True)

# check type of timestamp
print(load_per_year.dtypes)
print(load_per_year)

timestamp    datetime64[ns]
userId                int64
movieId               int64
rating                int64
dtype: object
    timestamp   userId  movieId   rating
0  1995-12-31        4        4        4
1  1996-12-31  1612609  1612609  1612609
2  1997-12-31   700982   700982   700982
3  1998-12-31   308070   308070   308070
4  1999-12-31  1198384  1198384  1198384
5  2000-12-31  1953659  1953659  1953659
6  2001-12-31  1186125  1186125  1186125
7  2002-12-31   869719   869719   869719
8  2003-12-31  1035878  1035878  1035878
9  2004-12-31  1170049  1170049  1170049
10 2005-12-31  1803158  1803158  1803158
11 2006-12-31  1171836  1171836  1171836
12 2007-12-31  1053430  1053430  1053430
13 2008-12-31  1158777  1158777  1158777
14 2009-12-31   930036   930036   930036
15 2010-12-31   903691   903691   903691
16 2011-12-31   766366   766366   766366
17 2012-12-31   731389   731389   731389
18 2013-12-31   599327   599327   599327
19 2014-12-31   562888   562888   562888
20 2015-12-31

In [6]:
# double check that 1995 really has only 4 entries in originial df
df[(df['timestamp'] > '1995-01-01 00:00:00') & (df['timestamp'] < '1996-01-01 00:00:00')]

,userId,movieId,rating,timestamp
4182421,28507,1176,4.0,1995-01-09 11:46:44
18950930,131160,21,3.0,1995-01-09 11:46:49
18950936,131160,47,5.0,1995-01-09 11:46:49
18950979,131160,1079,3.0,1995-01-09 11:46:49


In [7]:
# remove 1995 from load_per_year dataframe because it's confirmed to only have 4 entries
# and 2016 because that data was only collected until March 31,2015
load_per_year = load_per_year.iloc[1:20,:] # you used same variable to avoid slice on copy warning
load_per_year

,timestamp,userId,movieId,rating
1,1996-12-31,1612609,1612609,1612609
2,1997-12-31,700982,700982,700982
3,1998-12-31,308070,308070,308070
4,1999-12-31,1198384,1198384,1198384
5,2000-12-31,1953659,1953659,1953659
6,2001-12-31,1186125,1186125,1186125
7,2002-12-31,869719,869719,869719
8,2003-12-31,1035878,1035878,1035878
9,2004-12-31,1170049,1170049,1170049
10,2005-12-31,1803158,1803158,1803158


In [8]:
# convert timestamp back to string and only had the year label
load_per_year["timestamp"] = load_per_year["timestamp"].dt.strftime("%Y")

In [9]:
# check that the date is an object again
load_per_year.dtypes

timestamp    object
userId        int64
movieId       int64
rating        int64
dtype: object

In [10]:
######################################
# if you wanted to re-index the dataframe
#corrected_load_per_year.reset_index(inplace=True)
load_per_year

,timestamp,userId,movieId,rating
1,1996,1612609,1612609,1612609
2,1997,700982,700982,700982
3,1998,308070,308070,308070
4,1999,1198384,1198384,1198384
5,2000,1953659,1953659,1953659
6,2001,1186125,1186125,1186125
7,2002,869719,869719,869719
8,2003,1035878,1035878,1035878
9,2004,1170049,1170049,1170049
10,2005,1803158,1803158,1803158


In [11]:
# keep timestamp and one other columns for keeping track of the entries count
load_per_year.drop(['movieId', 'rating'], axis=1, inplace=True)
load_per_year

,timestamp,userId
1,1996,1612609
2,1997,700982
3,1998,308070
4,1999,1198384
5,2000,1953659
6,2001,1186125
7,2002,869719
8,2003,1035878
9,2004,1170049
10,2005,1803158


In [12]:
# rename time grouping and the other column to a count label
load_per_year.rename(columns = {"timestamp": "year collected", "userId": "number of entries"}, inplace=True)
load_per_year.dtypes

year collected       object
number of entries     int64
dtype: object

In [13]:
# final step is convert to list of dictionaries for the API. Call the list loads because your endpoint it /loads
loads = load_per_year.to_dict("records")
type(loads)

list

In [14]:
# calculate min,max,standard dev, and average and make sure they're all integers for the API
year_max = int(load_per_year.max()[1])

year_min = int(load_per_year.min()[1])

standard_dev_loads = int(load_per_year.std()[0])

average_load = int(load_per_year.mean()[1])
 

print(year_max)
print(year_min)
print(standard_dev_loads)
print(average_load)

type(year_min)

1953659
308070
417816
1037703


int

In [15]:
# make a dictionary of all the stats together
# use defaultdict to make an empty dict
from collections import defaultdict


statistics = defaultdict(dict)  # makes an empty dictionary
statistics["loads per year"] = loads
statistics["max load per year"] = year_max
statistics["min load per year"] = year_min
statistics["standard deviation of loads per year"] = standard_dev_loads
statistics["average loads per year"] = average_load

# Make API

In [16]:
from flask import Flask, jsonify, make_response

# create app from Flask class which gives each file a unique name
app = Flask(__name__) 

In [17]:
# Make request decorator that fires after the request is made so all requests get the the custom header
@app.after_request
def after_request(response):
    response.headers["X-Service_Version"] = "v0.1.0"
    return response

In [18]:
# GET /statistics
@app.route("/statistics") 
def get_statistics():
    return jsonify({"statistics": statistics})

In [19]:
# GET /statistics/loads
@app.route("/statistics/loads") 
def get_statistics_loads():
    return jsonify({"loads per year": loads})

In [20]:
# GET /statistics/loads/avg
@app.route("/statistics/loads/avg")
def get_statistics_avg():
    return jsonify({"average loads per year": average_load})

In [21]:
# GET /statistics/loads/stddev
@app.route("/statistics/loads/stddev") 
def get_statistics_sttdev():
    return jsonify({"standard deviation of loads per year": standard_dev_loads})

In [22]:
# GET /statistics/loads/min
@app.route("/statistics/loads/min")
def get_statistics_min():
    return jsonify({"min load per year": year_min})

In [23]:
# GET /statistics/loads/max
@app.route("/statistics/loads/max")
def get_statistics_max():
    return jsonify({"max per year": year_max})

In [ ]:
# GET /statistics/loads/<string:name>   # returns message that the desired calculation does not exist
@app.route('/statistics/loads/<string:name>')
def get_not_performed_message(name):
    if name not in ("loads", "avg", "min", "max"):
        return jsonify({"message": "calculation not performed"})

In [ ]:
app.run(port=5000)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
